In [1]:
from marshmallow import Schema, fields, post_load
from datetime import date, datetime

### create object classes

In [2]:
class Inner:
    def __init__(self, name, title, password):
        self.name = name
        self.title = title
        self.password = password
    
    def __repr__(self):
        return f"<Inner(name={self.name})>"

    
class Outer:
    def __init__(self, name, inner):
        self.name = name
        self.inner = inner
        self.created = datetime.now()
    
    def __repr__(self):
        return f"<Outer({self.name}, inner={self.inner.name}>"

In [3]:
in_obj = Inner('inName', 'Title', '1234')
out_obj = Outer('outName', in_obj)

In [4]:
out_obj.created

datetime.datetime(2019, 7, 29, 22, 46, 38, 172715)

### create Schemas

In [5]:
class InnerSchema(Schema):
    name = fields.Str()
    title = fields.Str()
    password = fields.Str()
    
    @post_load
    def make_inner(self, data, **kwargs):
        return Inner(**data)
    
class OuterSchema(Schema):
    name = fields.Str()
    created = fields.DateTime()
    inner = fields.Nested(InnerSchema())
    
    @post_load
    def make_outer(self, data, **kwargs):
        inner_obj = InnerSchema().load(data['inner'])
        print(inner_obj)
        return Outer(data['name'], inner_obj)
        
    
inner_schema = InnerSchema()
outer_schema = OuterSchema()

In [6]:
in_dict = inner_schema.dump(in_obj)
in_dict
test = inner_schema.load(in_dict)
test.password

{'name': 'inName', 'title': 'Title', 'password': '1234'}

'1234'

In [7]:
out_dict = outer_schema.dump(out_obj)
out_dict
out_test = outer_schema.load(out_dict)

{'name': 'outName',
 'inner': {'name': 'inName', 'title': 'Title', 'password': '1234'},
 'created': '2019-07-29T22:46:38.172715+00:00'}

ValidationError: {'_schema': ['Invalid input type.']}

In [13]:
my_object = outer_schema.load(out_dict)

In [14]:
my_object

{'inner': {'password': '1234', 'name': 'inName', 'title': 'Title'},
 'created': datetime.datetime(2019, 7, 26, 15, 2, 32, 879206, tzinfo=datetime.timezone(datetime.timedelta(0), '+0000')),
 'name': 'outName'}